# Pixel induced current
In this notebook we load a track dataset and we calculate the current induced on the pixels placed at the anode.

In [15]:
# This is need so you can import larndsim without doing python setup.py install
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [16]:
from larndsim import detsim

import importlib
importlib.reload(detsim)
from larndsim import consts, quenching, drifting
importlib.reload(drifting)
importlib.reload(quenching)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import pickle
import numpy as np
import numba as nb
from tqdm import tqdm_notebook as progress_bar
import pandas as pd

Here we load the dataset with the tracks and we store the column cols corresponding to the each variable.

In [17]:
tracks = pickle.load(open('edepsimDataset.p', 'rb'))
tracks = pd.DataFrame.from_dict(tracks)
tracks['t'] = 0.
tracks['t_start'] = 0.
tracks['t_end'] = 0.
tracks['x_start'] /= 1e2
tracks['y_start'] /= 1e2
tracks['z_start'] /= 1e2
tracks['x_end'] /= 1e2
tracks['y_end'] /= 1e2
tracks['z_end'] /= 1e2
tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) +
                    pow(tracks['y_end']-tracks['y_start'], 2) +
                    pow(tracks['z_end']-tracks['z_start'], 2))
tracks['x'] = (tracks['x_end']+tracks['x_start'])/2
tracks['y'] = (tracks['y_end']+tracks['y_start'])/2
tracks['z'] = (tracks['z_end']+tracks['z_start'])/2
tracks['dE'] = np.abs(tracks['dE'])*1e3
tracks['dEdx'] = tracks['dE']/tracks['dx']
tracks['NElectrons'] = 0.
tracks['longDiff'] = 0.
tracks['tranDiff'] = 0.

Here we pass our columnar data through the quenching and drifting stages, so we calculate the amount of electrons that reach the anode.

In [18]:
tracks_np = tracks.to_numpy()
cols = nb.typed.Dict()
for i, c, in enumerate(tracks.columns):
    cols[c] = i

In [19]:
quenching.Quench(tracks_np, cols, mode="box")

In [20]:
drifting.Drift(tracks_np, cols)

In [21]:
selectedTracks = tracks_np[:100]

Here we instantiate a TPC class with a determined number of pixels per axis and a determined sampling in time.

In [22]:
tpc = detsim.TPC(cols, t_sampling=0.5, n_pixels=400)

## Charge distribution calculation
Here we calculate the current induced by each track on the pixels, taking into account longitudinal and transverse diffusion. The track segment is parametrized as:
\begin{align}
x'(r') &=x_s + \frac{\Delta x}{\Delta r}r'\\
y'(r') &=y_s + \frac{\Delta y}{\Delta r}r'\\
z'(r') &=z_s + \frac{\Delta z}{\Delta r}r',
\end{align}
where $\Delta r$ is the segment length. Here we assume $z$ as the drift direction.
The diffused charge distribution is calculated with the following integral:
\begin{equation}
\rho(x,y,z) = \frac{Q}{\sqrt{(2\pi)^3}\sigma_x\sigma_y\sigma_z\Delta r}\exp\left[-\frac{(x-x_s)^2}{2\sigma_x^2}-\frac{(y-y_s)^2}{2\sigma_y^2}-\frac{(z-z_s)^2}{2\sigma_z^2}\right]\int^{r'=\Delta r}_{r'=0}dr'\exp[-(ar'^2+br')],
\end{equation}
where 
\begin{align}
a &= \left[\left(\frac{\Delta x}{\Delta r}\right)^2\frac{1}{2\sigma_x^2} + \left(\frac{\Delta y}{\Delta r}\right)^2\frac{1}{2\sigma_y^2} + \left(\frac{\Delta z}{\Delta r}\right)^2\frac{1}{2\sigma_z^2} \right]\\
b &= -\left[\frac{(x-x_s)}{\sigma_x^2}\frac{\Delta x}{\Delta r}+
\frac{(y-y_s)}{\sigma_y^2}\frac{\Delta y}{\Delta r}+
\frac{(z-z_s)}{\sigma_z^2}\frac{\Delta z}{\Delta r}\right].
\end{align}

The simmetry of the transverse diffusion along the track allows to take a slice on the $xy$ plane and solve the integral once at a fixed $z$ coordinate (e.g. at $z_{m} = (z_s+z_e)/2$) and re-use it at other $z$ coordinates away from the endpoints (where $\rho(x,y,z)$ varies along $z$ so must be calculated at each $z$). 

In [23]:
for t in progress_bar(selectedTracks, desc='Processing tracks...'):
    img = tpc.calculate_current(t)

In [31]:
selectedTracks[:,cols['_start']]

array([5760.56000501, 5885.10011504, 6120.91890024, 5933.92017016,
       5750.8195155 , 5733.64229597, 5734.17367862, 5734.58292729,
       5734.91635925, 5735.2029717 , 5735.39808304, 5735.53553317,
       5735.65883259, 5733.64229612, 5734.94773037, 5735.97809918,
       5736.79458501, 5737.46602159, 5737.991295  , 5738.41360203,
       5738.74496734, 5739.61032401, 5733.64229577, 5733.10834937,
       5732.6859903 , 5732.34096502, 5732.04582156, 5731.82555728,
       5731.64767178, 5733.64229736, 5741.52506033, 5742.75511152,
       5748.72158465, 5753.44223954, 5757.21951928, 5760.32134666,
       5762.77461926, 5764.74495992, 5766.34919079, 5767.66899391,
       5768.74779973, 5769.59244797, 5770.25071203, 5770.7808421 ,
       5771.2013472 , 5771.5360487 , 5733.6422958 , 5733.24463348,
       5732.6812797 , 5732.45528526, 5732.71491672, 5733.18283231,
       5733.41504703, 5733.53605784, 5733.62940727, 5733.74917573,
       5733.85113174, 5733.9083291 , 5733.89709841, 5733.96770

## Track and pixel display
Here we show the selected tracks and the pixels affected by them. The color of the pixel correspond to the integrated induced current.

In [24]:
%matplotlib widget
fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot(111, projection='3d')

currents = np.array([sum(tpc.pixel_response(p)) for p in tpc.active_pixels])
norm = mpl.colors.LogNorm(vmin=min(currents[currents>0]), vmax=max(currents))
cmap = cm.viridis
m = cm.ScalarMappable(norm=norm, cmap=cmap)


for i,p in enumerate(list(tpc.active_pixels.keys())):
    rect = plt.Rectangle((p[0]*tpc.x_pixel_size+consts.tpcBorders[0][0],
                          p[1]*tpc.y_pixel_size+consts.tpcBorders[1][0]),
                         tpc.x_pixel_size,tpc.y_pixel_size,
                         linewidth=0,
                         edgecolor='grey', 
                         facecolor=m.to_rgba(currents[i]), label='Pixel')
    ax.add_patch(rect)
    art3d.pathpatch_2d_to_3d(rect, z=-50, zdir="z")
    
for t in selectedTracks:
    ax.plot((t[cols['x_start']], t[cols['x_end']]), 
            (t[cols['y_start']], t[cols['y_end']]),
            (t[cols['z_start']], t[cols['z_end']]),
            c='r',
            zorder=99999)
    ax.scatter((t[cols['x_start']], t[cols['x_end']]), 
               (t[cols['y_start']], t[cols['y_end']]),
               (t[cols['z_start']], t[cols['z_end']]),
               c='r')

ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_zlabel("$z$ coordinate [cm]")
ax.set_zlim3d(-50,150)
ax.set_xlim3d(-5,5)
ax.set_ylim3d(-7,7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: operands could not be broadcast together with shapes (0,) (80,) (0,) 

In [25]:
%matplotlib widget
fig,ax = plt.subplots(1,1, constrained_layout=True)
currents = np.array([sum(tpc.pixel_response(p)) for p in tpc.active_pixels])

for i,p in enumerate(list(tpc.active_pixels.keys())):
    if currents[i]:
        rect = plt.Rectangle((p[0]*tpc.x_pixel_size+consts.tpcBorders[0][0],
                              p[1]*tpc.y_pixel_size+consts.tpcBorders[1][0]),
                             tpc.x_pixel_size,tpc.y_pixel_size,
                             linewidth=0.3,
                             edgecolor='grey', 
                             facecolor=m.to_rgba(currents[i]), label='Pixel')
        ax.add_patch(rect)
    
for t in selectedTracks:
    ax.plot((t[cols['x_start']], t[cols['x_end']]), 
            (t[cols['y_start']], t[cols['y_end']]),
            c='r',
            zorder=10)
    ax.scatter((t[cols['x_start']], t[cols['x_end']]), 
               (t[cols['y_start']], t[cols['y_end']]),
               c='r',
               zorder=10)
# ax.set_xlim(0,5)
# ax.set_ylim(-7,2)
ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_aspect("equal")
_ = plt.colorbar(m,fraction=0.046, pad=0.04, label='Current integral [a.u.]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: operands could not be broadcast together with shapes (0,) (80,) (0,) 

In [14]:
fig, ax = plt.subplots(1,1, constrained_layout=True)
pixelID = tpc.pixel_from_coordinates(2.14,0.84)
current = tpc.pixel_response(pixelID)
ax.plot(tpc.anode_t, current, label='Current')
ax.set_xlim(800,900)  
ax.set_title("Pixel (%i,%i) at (%.2f, %2.f)" % (pixelID[0], pixelID[1], 
                                      (pixelID[0] + 0.5)*tpc.x_pixel_size+consts.tpcBorders[0][0], 
                                      (pixelID[1] + 0.5)*tpc.y_pixel_size+consts.tpcBorders[1][0]))
ax.set_xlabel("Time [$\mathrm{\mu}$s]")
ax.set_ylabel("Current [a.u.]")
# ax.set_yscale("log")
# ax.set_ylim(1,1e7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …